# COMS 4995 HW4
#### Jianfeng Zhuang: JZ3872
#### Huazhang Liu: HL3338

### Task 3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import randint
import spacy
from transformers import pipeline

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import Ridge

#### Doing the same preprocessing like in Task 1 first

In [3]:
wine = pd.read_csv("winemag-data-130k-v2.csv")
wine = wine.drop("Unnamed: 0", axis = 1)
# wine.head()

In [4]:
wine_US = wine[wine['country'] == "US"]
wine_US = wine_US.drop('country', axis = 1)
wine_US.index = np.arange(0, wine_US.shape[0])
# wine_US

In [5]:
wine_US['points'] = wine_US['points'].astype('float64')
wine_US_1 = wine_US.copy()
wine_US_1.head()

,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,"Tart and snappy, the flavors of lime flesh and...",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87.0,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
4,"Slightly reduced, this wine offers a chalky, t...",NaN,87.0,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini


In [6]:
wine_US_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54504 entries, 0 to 54503
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   description            54504 non-null  object 
 1   designation            36908 non-null  object 
 2   points                 54504 non-null  float64
 3   price                  54265 non-null  float64
 4   province               54504 non-null  object 
 5   region_1               54226 non-null  object 
 6   region_2               50511 non-null  object 
 7   taster_name            37730 non-null  object 
 8   taster_twitter_handle  34741 non-null  object 
 9   title                  54504 non-null  object 
 10  variety                54504 non-null  object 
 11  winery                 54504 non-null  object 
dtypes: float64(2), object(10)
memory usage: 5.4+ MB


In [7]:
wine_US_1['points'] = wine_US_1['points'].astype('float64')
wine_US_1 = wine_US_1[(wine_US_1['price']!= 2013) & (wine_US_1['price'] != 750)]

In [8]:
wine_US_1 = wine_US_1[pd.isna(wine_US_1['taster_name']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['description']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['designation']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['region_1']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['region_2']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['title']) == False]
sample_wine_US_1 = wine_US_1.sample(10000)
sample_wine_US_1.index = np.arange(0, sample_wine_US_1.shape[0])
wine_US_1_text = sample_wine_US_1.drop(['price', 'province', 'taster_name', 'taster_twitter_handle', 'variety', 'winery'], axis = 1)
wine_US_1_text.head()

,description,designation,points,region_1,region_2,title
0,Dark chocolate and black cherry converge for a...,Estate Reserve,91.0,Paso Robles,Central Coast,San Simeon 2013 Estate Reserve Cabernet Sauvig...
1,Tons of spice notes entertain the nose on this...,Estate Grown,92.0,Santa Ynez Valley,Central Coast,Buttonwood 2013 Estate Grown Cabernet Franc (S...
2,Equal parts Cabernet Sauvignon and Petit Verdo...,Reserve,94.0,Diamond Mountain District,Napa,Von Strasser 2012 Reserve Red (Diamond Mountai...
3,Prolific winemaker Andrew Murray crafted this ...,16 Crows Cuvée,88.0,Santa Ynez Valley,Central Coast,Highland Oaks 2011 16 Crows Cuvée Grenache-Syr...
4,This is a mix of 56% Sagemoor and 41% Champoux...,Old Vines Dedication Series #26,94.0,Columbia Valley (WA),Columbia Valley,Woodward Canyon 2006 Old Vines Dedication Seri...


In [9]:
X = wine_US_1_text.drop('points', axis = 1)
y = wine_US_1_text['points']
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.head()

,description,designation,region_1,region_2,title
7649,"A 50/50 blend of Pommard and Dijon 777 grapes,...",Mark VII,Chehalem Mountains,Willamette Valley,Anam Cara 2012 Mark VII Pinot Noir (Chehalem M...
9592,A whiff of spring blossoms introduces this lus...,Stony Lonesome Estate Reserve Dry Rosé of,Finger Lakes,Finger Lakes,Three Brothers 2016 Stony Lonesome Estate Rese...
5040,"Aromas of purple fruit, cocoa, green-pepper, v...",Estate Grown,Columbia Valley (WA),Columbia Valley,Gamache 2012 Estate Grown Cabernet Sauvignon (...
5170,From a hilltop spot in an appellation increasi...,Bonnie Glen,Bennett Valley,Sonoma,MacLaren 2012 Bonnie Glen Syrah (Bennett Valley)
6292,Winemaker Bill Brousseau taps his family viney...,Brosseau Vineyard,Chalone,Central Coast,Testarossa 2014 Brosseau Vineyard Chardonnay (...


#### Using SpaCY Word2Vec model here

In [10]:
# load pipeline object
nlp = pipeline("feature-extraction")

In [11]:
region1_pipe = np.vstack([nlp(doc.encode().decode()) for doc in X_train['region_1']])

KeyboardInterrupt: 

In [13]:
# transform each sentence into vector and make the word2vec matrix
descript2vec = np.vstack([nlp(doc.encode().decode()).vector for doc in X_train['description']])
design2vec = np.vstack([nlp(doc.encode().decode()).vector for doc in X_train['designation']])
region12vec = np.vstack([nlp(doc.encode().decode()).vector for doc in X_train['region_1']])
region22vec = np.vstack([nlp(doc.encode().decode()).vector for doc in X_train['region_2']])
title2vec = np.vstack([nlp(doc.encode().decode()).vector for doc in X_train['title']])

In [14]:
# combine all matrix into one horizontally
X_train2vec = np.concatenate((descript2vec, design2vec, region12vec, region22vec, title2vec), axis = 1)
X_train2vec.shape

(7500, 1500)

In [15]:
Ridge2vec_score = cross_val_score(Ridge(), X_train2vec, np.ravel(y_train), cv = 5)
print('The result of transfering all text column into vector by using Word2Vec is', np.mean(Ridge2vec_score))

The result of transfering all text column into vector by using Word2Vec is 0.5288328995789419


In [31]:
# Here to remind the model we have for text features using CountVectorizer in Task 1
preprocess_task1 = make_column_transformer((CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b"), 'description'),
                                       (CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b"), 'designation'),
                                       (CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b"), 'region_1'),
                                       (CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b"), 'region_2'),
                                       (CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b"), 'title'))
task1_model = make_pipeline(preprocess_task1, Ridge())
task1_score = cross_val_score(task1_model, X_train, np.ravel(y_train), cv = 5)
print('The result of using bag of word only in task 1 is', np.mean(task1_score))

The result of using bag of word only in task 1 is 0.6426178023915253


#### Here we consider all five text features in word2vec embedding. Compared with the result in task 1 for the text features only using Bag of Words, the score of Word2Vec is lower. So using Word2Vec only does not improve the regression

In [16]:
bow_model = CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b")

In [17]:
# use CountVectorizer to transform each text feature in order to combine them with word2vec matrix
descript_bow = bow_model.fit_transform(X_train['description']).toarray()
design_bow = bow_model.fit_transform(X_train['designation']).toarray()
region1_bow = bow_model.fit_transform(X_train['region_1']).toarray()
region2_bow = bow_model.fit_transform(X_train['region_2']).toarray()
title_bow = bow_model.fit_transform(X_train['title']).toarray()

#### Since Word2Vec consider the semantics of the sentences but bag of word does not consider semanticsm, only consider the occurence of the word, here we use Word2Vec in description, designation, and title. For region 1 and region 2, we think the occurence is more important so we choose to use bag of word. 

In [21]:
X_train_nlp_1 = np.concatenate((descript2vec, design2vec, region1_bow, region2_bow, title2vec), axis = 1)
Ridge_nlp_score_1 = cross_val_score(Ridge(), X_train_nlp_1, np.ravel(y_train), cv = 5)
print('The result of X_train_nlp_1 is', np.mean(Ridge_nlp_score_1))

The result of X_train_nlp_1 is 0.5298221580511185


#### The score is higher than Ridge2vec_score Word2Vec only. 

#### Now we consider which one to use for designation:

In [37]:
X_train_nlp_2 = np.concatenate((descript2vec, design_bow, region1_bow, region2_bow, title2vec), axis = 1)
Ridge_nlp_score_2 = cross_val_score(Ridge(), X_train_nlp_2, np.ravel(y_train), cv = 5)
print('The result of X_train_nlp_2 for designation bow is', np.mean(Ridge_nlp_score_2))

The result of X_train_nlp_2 for designation bow is 0.5115607886250462


In [38]:
X_train_nlp_3 = np.concatenate((descript2vec, design2vec, design_bow, region1_bow, region2_bow, title2vec), axis = 1)
Ridge_nlp_score_3 = cross_val_score(Ridge(), X_train_nlp_3, np.ravel(y_train), cv = 5)
print('The result of X_train_nlp_3 for designation both s', np.mean(Ridge_nlp_score_3))

The result of X_train_nlp_3 for designation both s 0.4939999464644111


#### Clearly, using both embedding methods for designations doesn't make sense. We would you Word2Vec for designation. 

#### Then, we are testing on which methods to use for title:

In [39]:
X_train_nlp_4 = np.concatenate((descript2vec, design2vec, region1_bow, region2_bow, title_bow), axis = 1)
Ridge_nlp_score_4 = cross_val_score(Ridge(), X_train_nlp_4, np.ravel(y_train), cv = 5)
print('The result of X_train_nlp_4 title in bow is', np.mean(Ridge_nlp_score_4))

The result of X_train_nlp_4 title in bow is 0.5386498533933974


In [28]:
X_train_nlp_5 = np.concatenate((descript2vec, design2vec, region1_bow, region2_bow, title2vec, title_bow), axis = 1)
Ridge_nlp_score_5 = cross_val_score(Ridge(), X_train_nlp_5, np.ravel(y_train), cv = 5)
print('The result of X_train_nlp_5 for title both is', np.mean(Ridge_nlp_score_5))

The result of X_train_nlp_5 for title both is 0.5385710149109275


#### The best method are using bag of words only although the difference is really small

#### In addition, we think it is necessary to consider description in both case since the corpus size for description is huge, so the occurance and the semantics are both important. 

In [29]:
X_train_nlp_6 = np.concatenate((descript2vec, descript_bow, design2vec, region1_bow, region2_bow, title_bow), axis = 1)
Ridge_nlp_score_6 = cross_val_score(Ridge(), X_train_nlp_6, np.ravel(y_train), cv = 5)
print('The result of X_train_nlp_6 for description both is', np.mean(Ridge_nlp_score_6))

The result of X_train_nlp_6 for description both is 0.6543457846631455


#### Lastly, we consider all of the features by using both method.

In [32]:
X_train_nlp_all = np.concatenate((X_train2vec, descript_bow, design_bow, 
                                  region1_bow, region2_bow, title_bow), axis = 1)
X_train_nlp_all.shape

(7500, 382752)

In [33]:
Ridge_nlp_score_all = cross_val_score(Ridge(), X_train_nlp_all, np.ravel(y_train), cv = 5)
print('The result of Ridge_nlp_score_all is', np.mean(Ridge_nlp_score_all))

The result of Ridge_nlp_score_all is 0.6555141027037592


#### Using both method on all of the features does not make huge difference between Ridge_nlp_score_6 because in both cases we consider description in both methods, which drive the results. 

In [35]:
Ridge_nlp_score_6_regularize = cross_val_score(Ridge(alpha = 0.0002), X_train_nlp_6, np.ravel(y_train), cv = 5)
print('The result of Ridge_nlp_score_6_regularize with alpha 0.0002 is', np.mean(Ridge_nlp_score_6_regularize))

The result of Ridge_nlp_score_6_regularize with alpha 0.0002 is 0.6542919657987897


#### To prevent overfitting, we apply alpha on X_train_nlp_6, but the result is very close. 